In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import csv
import os
import pandas as pd
import numpy as np

# User details and dataset file paths
user_details_path = '/content/drive/My Drive/IR_Project/user_details.csv'
gym_dataset_path = '/content/drive/My Drive/IR_Project/TrainingDataset.csv'

# Welcome message
print("Welcome to the Training Program Recommender System!")
print("Your journey towards a personalized fitness experience begins here.\n")

# Initialize or load the users DataFrame
if os.path.exists(user_details_path):
    users_df = pd.read_csv(user_details_path)
else:
    users_df = pd.DataFrame(columns=['Username', 'Password'])

def register():
    global users_df  # Ensure we're modifying the global DataFrame
    username = input("Enter a new username: ")
    if username in users_df['Username'].values:
        print("Username already exists. Please try a different username or type 'back' to return to the previous menu.")
        return False  # Exit the function if username exists or if 'back' is typed

    password = input("Enter password: ")
    # Append the new user details to the DataFrame
    new_user = pd.DataFrame([[username, password]], columns=['Username', 'Password'])
    users_df = pd.concat([users_df, new_user], ignore_index=True)

    # Save the updated DataFrame back to the CSV to persist changes
    users_df.to_csv(user_details_path, index=False)
    print("Registration successful. You can now login with your new credentials.\n")
    return True


def login():
    while True:
        username = input("Enter username: ")
        password = input("Enter password: ")
        if username in users_df['Username'].values and password == users_df.loc[users_df['Username'] == username, 'Password'].values[0]:
            print("Login successful. Welcome back!\n")
            return True
        else:
            print("Invalid username or password. Please try again or type 'back' to return to the main menu.")
            if input().lower() == 'back':
                return False

def user_auth():
    while True:
        choice = input("Do you want to login or register? (login/register): ").lower()
        if choice == 'login':
            if login():
                return True
        elif choice == 'register':
            if register():
                continue
        else:
            print("Invalid option. Please type 'login' or 'register'.")



def calculate_evaluation_metrics(user_ratings):
    relevant_ratings = [rating for rating in user_ratings if rating >= 8]  # Adjust threshold as needed
    not_relevant_ratings = [rating for rating in user_ratings if rating < 8]

    true_positives = len(relevant_ratings)
    false_positives = len(not_relevant_ratings)
    false_negatives = 0  # Simplified for this example

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nEvaluation Metrics:\nPrecision: {precision:.2f}\nRecall: {recall:.2f}\nF1 Score: {f1_score:.2f}")

def get_user_ratings(recommended_exercises):
    print("\nPlease rate the following exercises on a scale of 1-10 for relevance to your goals:")
    ratings = []
    for _, row in recommended_exercises.iterrows():
        while True:
            try:
                rating = float(input(f"{row['Title']} (Rating 1-10): "))
                if rating < 1 or rating > 10:
                    raise ValueError
                ratings.append(rating)
                break
            except ValueError:
                print("Invalid rating. Please enter a value between 1 and 10 with up to one decimal point.")
    return ratings
def main():
    if not user_auth():
        return

    print("Let's gather some information about your training goals to provide personalized recommendations.")

    # Available body parts and equipment options
    available_body_parts = ['Abductors', 'Abdominals', 'Biceps', 'Triceps', 'Chest', 'Calves',
                            'Forearms', 'Glutes', 'Hamstrings', 'Lats', 'Lower back', 'Middle back',
                            'Traps', 'Neck', 'Quadriceps', 'Shoulders']

    available_equipment = ['Cable', 'Body Only', 'E-Z Curl Bar', 'Machine', 'Dumbbell', 'Bands', 'Kettlebells', 'Barbell', 'Other']

    # Display available options for body parts
    print("Available options for body parts:")
    print(', '.join(available_body_parts))
    body_part = input("Which body part do you want to focus on? ")
    while body_part.title() not in available_body_parts:
        print("Invalid body part. Please choose from the available options.")
        body_part = input("Which body part do you want to focus on? ")

    # Display available options for equipment
    print("\nAvailable options for equipment:")
    print(', '.join(available_equipment))
    equipment = input("What equipment do you have access to? ")
    while equipment.title() not in available_equipment:
        print("Invalid equipment. Please choose from the available options.")
        equipment = input("What equipment do you have access to? ")

    gym_dataset = pd.read_csv(gym_dataset_path)
    all_recommended_exercises = gym_dataset[(gym_dataset['BodyPart'].str.lower().str.contains(body_part.lower())) &
                                            (gym_dataset['Equipment'].str.lower().str.contains(equipment.lower()))]

    if all_recommended_exercises.empty:
        print("\nNo exercises found matching your criteria. Try different inputs.")
        return

    top_exercises = all_recommended_exercises.sort_values(by='Rating', ascending=False).head(3)
    print("\nBased on your inputs, here are the top 3 rated exercises:")
    print(top_exercises[['Title', 'Desc']])

    user_ratings = get_user_ratings(top_exercises)

    if np.mean(user_ratings) < 6:
        print("\nLet's explore more exercises that might suit your preferences better.")
        other_exercises = all_recommended_exercises.drop(top_exercises.index).sort_values(by='Rating', ascending=False).head(3)
        if not other_exercises.empty:
            print("Here are some other high-rated exercises:")
            print(other_exercises[['Title', 'Desc']])
            get_user_ratings(other_exercises)
        else:
            print("There are no more exercises to recommend based on your criteria.")
    else:
        calculate_evaluation_metrics(user_ratings)

if __name__ == "__main__":
    main()





Welcome to the Training Program Recommender System!
Your journey towards a personalized fitness experience begins here.

Do you want to login or register? (login/register): register
Enter a new username: Rahul
Enter password: 09876
Registration successful. You can now login with your new credentials.

Do you want to login or register? (login/register): login
Enter username: Rahul
Enter password: 09876
Login successful. Welcome back!

Let's gather some information about your training goals to provide personalized recommendations.
Available options for body parts:
Abductors, Abdominals, Biceps, Triceps, Chest, Calves, Forearms, Glutes, Hamstrings, Lats, Lower back, Middle back, Traps, Neck, Quadriceps, Shoulders
Which body part do you want to focus on? biceps

Available options for equipment:
Cable, Body Only, E-Z Curl Bar, Machine, Dumbbell, Bands, Kettlebells, Barbell, Other
What equipment do you have access to? dumbbell

Based on your inputs, here are the top 3 rated exercises:
      